LCELは、基本コンポーネントから複雑なチェーンを容易に構築できるようにし、ストリーミング、並列処理、ログ記録などの即座に利用可能な機能をサポートしています。> LCEL makes it easy to build complex chains from basic components, and supports out of the box functionality such as streaming, parallelism, and logging.

## Basic example: prompt + model + output parser | 基本例：プロンプト + モデル + 出力パーサー最も基本的でありながら一般的な使用例は、プロンプトテンプレートとモデルを組み合わせて連鎖させることです。この仕組みがどのように機能するかを確認するために、特定のトピックからジョークを生成するチェーンを作成してみましょう：> The most basic and common use case is chaining a prompt template and a model together. To see how this works, let's create a chain that takes a topic and generates a joke:

In [7]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

prompt = ChatPromptTemplate.from_template("tell me a short joke about {topic}")
model = ChatOpenAI()
output_parser = StrOutputParser()

chain = prompt | model | output_parser

chain.invoke({"topic": "ice cream"})

"Why did the ice cream go to therapy?\n\nBecause it had too many toppings and couldn't find its cone-fidence!"

このコードの行に注目してください。ここでは、LCELを使用して異なるコンポーネントを一つのチェーンに組み合わせています：> Notice this line of this code, where we piece together then different components into a single chain using LCEL:```chain = prompt | model | output_parser````|`記号は、[unix pipe operator](https://en.wikipedia.org/wiki/Pipeline_\(Unix\))に類似しており、異なるコンポーネントを連結し、一つのコンポーネントの出力を次のコンポーネントの入力へと渡す役割を果たします。> The `|` symbol is similar to a [unix pipe operator](https://en.wikipedia.org/wiki/Pipeline_\(Unix\)), which chains together the different components feeds the output from one component as input into the next component.このチェーンでは、ユーザーの入力がプロンプトテンプレートに渡され、その後プロンプトテンプレートからの出力がモデルに渡され、さらにモデルからの出力が出力パーサーに渡されます。各コンポーネントを個別に見ていくことで、実際に何が行われているのかを深く理解しましょう。> In this chain the user input is passed to the prompt template, then the prompt template output is passed to the model, then the model output is passed to the output parser. Let's take a look at each component individually to really understand what's going on.

### 1. Prompt | 1. プロンプト`prompt`は`BasePromptTemplate`の一種で、テンプレート変数の辞書を取り込んで`PromptValue`を生成します。`PromptValue`は、完成したプロンプトを包み込むラッパーで、文字列を入力とする`LLM`またはメッセージのシーケンスを入力とする`ChatModel`へ渡すことが可能です。`BaseMessage`を生成するためのロジックと文字列を生成するためのロジックの両方を定義しているので、どちらの言語モデルタイプにも適用可能です。> `prompt` is a `BasePromptTemplate`, which means it takes in a dictionary of template variables and produces a `PromptValue`. A `PromptValue` is a wrapper around a completed prompt that can be passed to either an `LLM` (which takes a string as input) or `ChatModel` (which takes a sequence of messages as input). It can work with either language model type because it defines logic both for producing `BaseMessage`s and for producing a string.

In [8]:
prompt_value = prompt.invoke({"topic": "ice cream"})
prompt_value

ChatPromptValue(messages=[HumanMessage(content='tell me a short joke about ice cream')])

In [9]:
prompt_value.to_messages()

[HumanMessage(content='tell me a short joke about ice cream')]

In [10]:
prompt_value.to_string()

'Human: tell me a short joke about ice cream'

### 2. Model | 2. モデル`PromptValue`はその後`model`に渡されます。この場合、私たちの`model`は`ChatModel`であり、これは`BaseMessage`を出力するという意味です。> The `PromptValue` is then passed to `model`. In this case our `model` is a `ChatModel`, meaning it will output a `BaseMessage`.

In [11]:
message = model.invoke(prompt_value)
message

AIMessage(content="Why did the ice cream go to therapy? \n\nBecause it had too many toppings and couldn't find its cone-fidence!")

もし私たちの`model`が`LLM`であったなら、文字列を出力するでしょう。> If our `model` was an `LLM`, it would output a string.

In [12]:
from langchain.llms import OpenAI

llm = OpenAI(model="gpt-3.5-turbo-instruct")
llm.invoke(prompt_value)

'\n\nRobot: Why did the ice cream go to therapy? Because it had a rocky road.'

### 3. Output parser | 3. 出力パーサー最後に、私たちは`model`の出力を`output_parser`に渡します。これは`BaseOutputParser`であり、文字列または`BaseMessage`を入力として受け取ることを意味しています。具体的には、`StrOutputParser`はどんな入力も単純に文字列に変換します。> And lastly we pass our `model` output to the `output_parser`, which is a `BaseOutputParser` meaning it takes either a string or a> `BaseMessage` as input. The `StrOutputParser` specifically simple converts any input into a string.

In [13]:
output_parser.invoke(message)

"Why did the ice cream go to therapy? \n\nBecause it had too many toppings and couldn't find its cone-fidence!"

### 4. Entire Pipeline | 4. 全体のパイプライン手順に従って進めます：> To follow the steps along:1. ユーザーが望むトピックに関する入力を `{"topic": "ice cream"}` として渡します   > We pass in user input on the desired topic as `{"topic": "ice cream"}`2. `prompt`コンポーネントはユーザーの入力を受け取り、その入力の`topic`を使用してプロンプトを構築した後、PromptValueを生成します。   > The `prompt` component takes the user input, which is then used to construct a PromptValue after using the `topic` to construct the prompt.3. `model` コンポーネントは生成されたプロンプトを受け取り、評価のために OpenAI LLM モデルに渡します。モデルから生成される出力は `ChatMessage` オブジェクトです。   > The `model` component takes the generated prompt, and passes into the OpenAI LLM model for evaluation. The generated output from the model is a `ChatMessage` object.4. 最後に、`output_parser` コンポーネントは `ChatMessage` を受け取り、それをPythonの文字列に変換して、invokeメソッドから返します。   > Finally, the `output_parser` component takes in a `ChatMessage`, and transforms this into a Python string, which is returned from the invoke method.

```mermaidgraph LR    A(Input: topic=ice cream) --> |Dict| B(PromptTemplate)    B -->|PromptValue| C(ChatModel)        C -->|ChatMessage| D(StrOutputParser)    D --> |String| F(Result)```

:::infoもしコンポーネントの出力について知りたい場合は、`prompt` や `prompt | model` のようなチェーンの簡易版をテストして、中間結果を確認することが常に可能です。> Note that if you’re curious about the output of any components, you can always test out a smaller version of the chain such as `prompt`  or `prompt | model` to see the intermediate results::::

In [ ]:
input = {"topic": "ice cream"}

prompt.invoke(input)
# > ChatPromptValue(messages=[HumanMessage(content='tell me a short joke about ice cream')])

(prompt | model).invoke(input)
# > AIMessage(content="Why did the ice cream go to therapy?\nBecause it had too many toppings and couldn't cone-trol itself!")

## RAG Search Example | RAG検索の例次の例で、私たちは質問に答える際に文脈を付加するために、検索強化生成チェーンを実行したいと考えています。> For our next example, we want to run a retrieval-augmented generation chain to add some context when responding to questions.

In [ ]:
# Requires:
# pip install langchain docarray tiktoken

from langchain.chat_models import ChatOpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain.prompts import ChatPromptTemplate
from langchain.vectorstores import DocArrayInMemorySearch
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

vectorstore = DocArrayInMemorySearch.from_texts(
    ["harrison worked at kensho", "bears like to eat honey"],
    embedding=OpenAIEmbeddings(),
)
retriever = vectorstore.as_retriever()

template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
model = ChatOpenAI()
output_parser = StrOutputParser()

setup_and_retrieval = RunnableParallel(
    {"context": retriever, "question": RunnablePassthrough()}
)
chain = setup_and_retrieval | prompt | model | output_parser

chain.invoke("where did harrison work?")

この場合、構成されたチェーンは以下の通りです：> In this case, the composed chain is:

In [ ]:
chain = setup_and_retrieval | prompt | model | output_parser

これを説明するには、まず上記のプロンプトテンプレートが`context`と`question`をプロンプト内で置換えられる値として取り込んでいることを確認します。プロンプトテンプレートを構築する前に、検索に関連した文書を取得し、それらをコンテキストの一部として組み込むことが望まれます。> To explain this, we first can see that the prompt template above takes in `context` and `question` as values to be substituted in the prompt. Before building the prompt template, we want to retrieve relevant documents to the search and include them as part of the context.予備段階として、クエリに基づいて文書を検索できるインメモリストアを使用してリトリバーを設定しました。これは他のコンポーネントと連携してチェーンすることが可能な実行可能なコンポーネントですが、単独で実行することも試みることができます。> As a preliminary step, we’ve setup the retriever using an in memory store, which can retrieve documents based on a query. This is a runnable component as well that can be chained together with other components, but you can also try to run it separately:

In [ ]:
retriever.invoke("where did harrison work?")

次に、検索された文書のエントリと元のユーザーの質問を用い、プロンプトに予期される入力を準備するために`RunnableParallel`を使用します。このプロセスには、文書検索用のリトリバーと、ユーザーの質問を伝達する`RunnablePassthrough`の使用が含まれます：> We then use the `RunnableParallel` to prepare the expected inputs into the prompt by using the entries for the retrieved documents as well as the original user question, using the retriever for document search, and RunnablePassthrough to pass the user’s question:

In [ ]:
setup_and_retrieval = RunnableParallel(
    {"context": retriever, "question": RunnablePassthrough()}
)

復習すると、完全なチェーンは以下の通りです：> To review, the complete chain is:

In [ ]:
setup_and_retrieval = RunnableParallel(
    {"context": retriever, "question": RunnablePassthrough()}
)
chain = setup_and_retrieval | prompt | model | output_parser

フローは以下の通りです：> With the flow being:1. 最初のステップで、2つのエントリを持つ`RunnableParallel`オブジェクトが作成されます。最初のエントリ`context`には、リトリバーによって取得されたドキュメントの結果が含まれています。2番目のエントリ`question`には、ユーザーの元の質問が含まれています。質問を次のプロセスに渡すために、`RunnablePassthrough`を使用してこのエントリをコピーします。   > The first steps create a `RunnableParallel` object with two entries.  The first entry, `context` will include the document results fetched by the retriever. The second entry, `question` will contain the user’s original question. To pass on the question, we use `RunnablePassthrough` to copy this entry.2. 前の手順から得られた辞書を`prompt`コンポーネントに入力します。このコンポーネントは、ユーザーの入力である`question`と、取得されたドキュメントである`context`を取り込んでプロンプトを構築し、PromptValueを出力します。   > Feed the dictionary from the step above to the `prompt` component. It then takes the user input which is `question` as well as the retrieved document which is `context` to construct a prompt and output a PromptValue.3. `model` コンポーネントは生成されたプロンプトを受け取り、評価のために OpenAI LLM モデルに渡します。モデルから出力された結果は `ChatMessage` オブジェクトです。   > The `model` component takes the generated prompt, and passes into the OpenAI LLM model for evaluation. The generated output from the model is a `ChatMessage` object.4. 最終的に、`output_parser` コンポーネントは `ChatMessage` を受け取り、それをPythonの文字列に変換して、invokeメソッドからの戻り値として出力します。   > Finally, the `output_parser` component takes in a `ChatMessage`, and transforms this into a Python string, which is returned from the invoke method.```mermaidgraph LR    A(Question) --> B(RunnableParallel)    B -->|Question| C(Retriever)    B -->|Question| D(RunnablePassThrough)    C -->|context=retrieved docs| E(PromptTemplate)    D -->|question=Question| E    E -->|PromptValue| F(ChatModel)        F -->|ChatMessage| G(StrOutputParser)    G --> |String| H(Result)```

## Next steps | 次のステップ次に、LCELを使用した場合と使用しない場合で、共通機能を実現するために必要なコードを並べて比較する「[なぜLCELを使うのか](/docs/expression_language/why)」セクションをお読みいただくことをお勧めします。> We recommend reading our [Why use LCEL](/docs/expression_language/why) section next to see a side-by-side comparison of the code needed to produce common functionality with and without LCEL.